In [15]:
import numpy as np
from math import ceil
# Run the fitness 
from oct2py import Oct2Py
from multiprocessing.pool import ThreadPool 
# Run multiple Zs as different processes
from multiprocessing import Pool
# to save the progress of the algorithms
import csv
from datetime import datetime
from os import mkdir

In [2]:
# -------- range of the variables ----------
V_S = 7.0                   # service speed [kn]
range_D     = [0.5, 0.8]
range_AEdAO = [0.3, 1.05]
range_PdD   = [0.5, 1.4]
range_Z     = [2, 7]

# Define the lower and upper bounds for each variable
lower_bounds = [range_D[0], range_AEdAO[0], range_PdD[0]]
upper_bounds = [range_D[1], range_AEdAO[1], range_PdD[1]]

In [45]:
# size of step from each variable
LIST_SIZE = {'D':     int((range_D[1]-range_D[0])        / 0.05),
             'AEdAO': int((range_AEdAO[1]-range_AEdAO[0])/ 0.1),
             'PdD':   int((range_PdD[1]-range_PdD[0])    / 0.1)}
LIST_SIZE

{'D': 6, 'AEdAO': 7, 'PdD': 8}

In [46]:
t = [LIST_SIZE[v] for v in LIST_SIZE]
all_iter = t[0]*t[1]*t[2]
all_iter

336

In [17]:
# number of octave evaluations to run in parallel in each process
NUM_PARALLEL = 3

NUM_BATCHES = ceil(LIST_SIZE['PdD'] / NUM_PARALLEL)

3

In [6]:
# save the results, otherwise there is no output
save_file = True

In [7]:
def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
    
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    append_to_file(filename, row)
    
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]", 
                  "AEdAO = expanded area ratio", 
                  "PdD = pitch ratio", 
                  "Z = propeller's number of blades", 
                  "P_B = power brake", 
                  "n = Propeller angular speed [rpm]", 
                  "fitness", 
                  "i = thread pool id"]
        writer.writerow(header) 
    return filename

def create_dir(text):
    now = datetime.now()
    dir_name = './' + text +'_'+ now.strftime("%Y_%m_%d_%H_%M")
    try:
        mkdir(dir_name)
    except: pass
    return dir_name

In [19]:
def run_octave_evaluation(V_S,D,Z,AEdAO,PdD):
    P_B, n = [0, 0]
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        P_B, n = octave.F_LabH2_aprox(V_S,D,Z,AEdAO,PdD, nout=2)
    return [P_B, n]

Logic for the iterations

In [40]:
def wrapper(k, x):
    D, AEdAO, PdD = x
    P_B, n = run_octave_evaluation(V_S,D,Z,AEdAO,PdD)
    return [k, P_B]

def run_list_parallel(x_list):
    fitness_list = np.zeros(len(x_list))
    # run eval in parallel
    with ThreadPool(len(x_list)) as pool:
        id_x = [(k, x_list[k]) for k in range(len(x_list))]
        for result in pool.starmap(wrapper, id_x):
            k, fitness = result
            fitness_list[k] = fitness
    # go through the results and save
    for k in range(len(x_list)):
        P_B = fitness_list[k]
        D, AEdAO, PdD = x_list[k]
        if print_octave_result:
            print("D:",D,"Z:",Z,"AEdAO:",AEdAO,"PdD:",PdD, "P_B:",P_B)
        if save_file:
            append_to_file(filename, [D, AEdAO, PdD, Z, P_B])

In [41]:
list_D     = np.linspace(range_D[0],     range_D[1],     LIST_SIZE['D'])
list_AEdAO = np.linspace(range_AEdAO[0], range_AEdAO[1], LIST_SIZE['AEdAO'])
list_PdD   = np.linspace(range_PdD[0],   range_PdD[1],   LIST_SIZE['PdD'])

In [43]:
def run(z):
    global Z
    Z = z
    # create the csv file with the headers
    if save_file:
        global filename
        filename = create_file(str(Z))

    counter = 0
    for D in list_D:
        for AEdAO in list_AEdAO:
            # divide PdD in batches of size NUM_PARALLEL
            for b in range(NUM_BATCHES):
                start = b * NUM_PARALLEL
                end   = (b+1) * NUM_PARALLEL
                list_x = [(D,AEdAO,PdD) for PdD in list_PdD[start:end]]
                # run list in parallel
                run_list_parallel(list_x)
                # keep count of the progress
                counter += len(list_x)
                print('Z:',Z, counter,'/',all_iter)
                    
    return Z

parallel

In [ ]:
# create dir named 'gradient'
global dir_name
dir_name = create_dir('gradient')
    
if __name__ == '__main__':
    with Pool() as pool:
        for result_z in pool.map(run, range(range_Z[0],range_Z[1]+1)):
            print('end of ', result_z)

D:D:  0.50.5Z:   Z:5  4 AEdAO:AEdAO:  0.3 PdD: 0.3 0.5PdD: P_B:  0.50.0 P_B:
0.0D:  
0.5D:  Z:0.5  Z: 45  AEdAO:AEdAO: 0.3  0.3PdD:  0.6285714285714286PdD:  0.6285714285714286P_B:  0.0P_B:
 D:D:  0.5 0.00.5Z: 
2 D:  Z:AEdAO:0.5  0.3Z:   PdD: 5 0.5AEdAO: 0.3 4 P_B:PdD:  AEdAO: 0.7571428571428571 0.00.3D:  0.5 PdD: P_B:Z:
 D:3 0.5  Z:0.0
  AEdAO: 0.3 0.7571428571428571Z: 2 PdD:5 3 P_B: AEdAO: / 336   0.50.3
0.0  PdD:
P_B:  0.62857142857142860.0Z: P_B: 
D: 0.0
 4D:0.5   3Z:0.5   /Z:3  2336  
AEdAO:AEdAO: 0.3  0.3PdD: PdD: 0.7571428571428571  P_B:0.6285714285714286 0.0 
P_B: Z:0.0 2
 D:3  /0.5 Z: 336 
3 AEdAO: 0.3 PdD: 0.7571428571428571 P_B: 0.0
Z: 3 3 / 336
D:D: 0.5  0.5Z:  Z: 4 5AEdAO:  0.3AEdAO:  PdD:0.3 0.8857142857142857 PdD:  P_B:0.8857142857142857  0.0P_B:
 D:0.0 
0.5 D: Z:0.5  Z:4 5  AEdAO: AEdAO: 0.3 PdD:0.3  PdD:1.0142857142857142  1.0142857142857142P_B:  0.0P_B:
D: 0.0 
0.5D:  0.5Z:  Z:4  AEdAO:5  0.3 PdD:AEdAO: 0.3  PdD: 1.14285714285714281.1428571428571428  P_B:P_B: 0.0 0.0
Z

 D: 0.50.5 Z:  4Z:  3AEdAO:  AEdAO:0.8  0.8PdD: PdD:  0.6285714285714286 1.0142857142857142P_B: 0.0 P_B: 0.0
D: 0.5
 D:Z:  0.53  AEdAO:Z:  4 0.8AEdAO:  PdD:0.8  0.7571428571428571PdD:  1.1428571428571428P_B:  P_B:0.0 
0.0
Z: Z: 34  3942  //  336336

D: 0.5 Z: 2 AEdAO: 0.8 PdD: 0.5 P_B: 0.0
D: 0.5 Z: 2 AEdAO: D:0.8 0.5  PdD:Z:  0.62857142857142865 P_B:  AEdAO: 0.00.8
 PdD:D:  1.27142857142857140.5  P_B:Z:  0.02
 D:AEdAO: 0.8 0.5  PdD:Z: 0.7571428571428571  5P_B:  AEdAO:0.0
 Z:0.8 2  PdD: 391.4  /P_B:  0.0336
Z:
 5 45 / 336
D: 0.5 Z: 4 AEdAO: 0.8 PdD: 1.2714285714285714 P_B: 0.0
D: 0.5 Z: 4 AEdAO: 0.8 PdD: 1.4 P_B: 0.0
Z: 4 45 / 336
D: 0.5 Z: 3 AEdAO: 0.8 PdD: 0.8857142857142857 P_B: 0.0
D: 0.5 Z: 3 AEdAO: 0.8 PdD: 1.0142857142857142 P_B: 0.0
D: 0.5 Z: D: 30.5  Z: AEdAO:5 0.8 AEdAO:  PdD:0.925 1.1428571428571428  P_B:PdD:  0.50.0 
P_B:Z: 0.0 
3 D: 42 0.5/ 336 
Z: 5 D:AEdAO: 0.5 Z: 4 AEdAO: 0.925  0.925PdD:  PdD:0.6285714285714286  P_B:0.5  P_B:0.0
D: 0.5 Z: 0.0 
5D:  AEdAO:0.5 0.925 PdD:

results